<a href="https://colab.research.google.com/github/mingd00/aivle-miniproj6/blob/main/2%EC%B0%A8.%20%EC%9D%91%EA%B8%89%EC%83%81%ED%99%A9%20%EC%9D%B8%EC%8B%9D%20%EB%B0%8F%20%EC%9D%91%EA%B8%89%EC%8B%A4%20%EC%97%B0%EA%B3%84/1.%20%EC%9D%8C%EC%84%B1%20%EC%9D%B8%EC%8B%9D%20%EB%B0%8F%20%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **응급상황 자동 인식 및 응급실 연계 서비스**
### **응급상황 음성 인식 및 요약**

* 음성인식 : STT(Speech-to-Text)
    * 사용 모델 : OpenAI의 **Whisper-1**
* 텍스트 요약 및 핵심 키워드 도출
    * 사용 모델 : OpenAI의 **GPT-3.5-turbo**
    * 내용 요약과 주요 키워드를 도출

## **1.환경설정**

#### 1) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6_2/'

#### 2) 라이브러리

In [3]:
!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json

### (3) OpenAI API Key 환경 변수 설정

* open ai api key는는 **api_key.txt** 파일에 저장

In [5]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

## **2. STT**

* 음성파일 변환

In [9]:
# 음성파일 경로 지정
audio_path = path + 'audio/'

In [7]:
# OpenAI 클라이언트 생성
client = OpenAI()

In [ ]:
# 위스퍼 모델 사용 : 제공된 음성파일 중 1개를 텍스트로 변환 
filename = 'audio1.mp3'
audio_file = open(audio_path + filename, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
)

print(transcript, type(transcript))

지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.
 <class 'str'>


* 음성파일 변환 함수 생성

In [11]:
def audio_to_text(audio_path, filename):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text",
    )

    # 결과 반환
    return transcript

In [12]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

['audio4.mp3', 'audio5.mp3', 'audio3.mp3', 'audio1.mp3', 'audio2.mp3']


In [13]:
# 빈 데이터프레임 선언
df = pd.DataFrame(columns=['filename', 'text'])

# 반복문 수행하면서 오디오 변환
for filename in file_names:
  text = audio_to_text(audio_path, filename)
  df.loc[len(df)] = [filename, text]

# 데이터프레임 결과 조회
df

,filename,text
0,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
3,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
4,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...


- 나머지 오디오 데이터 변환

In [15]:
my_audio_path = path + 'my_audio/'
my_file_names = [f for f in os.listdir(my_audio_path) if os.path.isfile(os.path.join(my_audio_path, f))]
print(my_file_names)

['5등급(2).m4a', '5등급(1).m4a', '1등급(1).m4a', '2등급(1).m4a', '3등급(1).m4a', '2등급(2).m4a', '4등급(1).m4a', '3등급(2).m4a', '4등급(2).m4a', '1등급(2).m4a']


In [16]:
# 반복문 수행하면서 오디오 변환
for filename in my_file_names:
  text = audio_to_text(my_audio_path, filename)
  df.loc[len(df)] = [filename, text]

# 데이터프레임 결과 조회
df

,filename,text
0,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
3,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
4,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
5,5등급(2).m4a,저희 할머니가 삿고랑 부위에 통증과 함께 부종이 있다고 하세요. 상태가 심각해 보이...
6,5등급(1).m4a,제 조카가 벌에 쏘였는데요 소인 부위가 조금 붓고 아프다고 합니다. 숨 쉬는데 문제...
7,1등급(1).m4a,안녕하세요. 아버지가 방금 전 약을 잘못 드신 것 같아요. 숨쉬는 게 굉장히 힘들어...
8,2등급(1).m4a,제 어머니가 넘어지면서 눈을 다치셨어요. 부딪힌 후부터 시야가 흐릿해졌다고 하셔서 ...
9,3등급(1).m4a,저희 아버지가 갑자기 한쪽 귀가 안 들린다고 하세요. 어제까지는 괜찮으셨는데 오늘 ...


## **3. Summary**

* 문서 요약 예제

In [17]:
input_text = '''
한국은행 총재가 "올해 성장률이 기존 전망치 2.4%보다 낮아질 가능성이 크다"며 "2.2∼2.3% 정도로 떨어지지 않을까 생각한다"고 밝혔습니다.
이 총재는 오늘(29일) 국회 기획재정위원회 국정감사에 출석해 한은의 전망을 크게 밑돈 3분기 성장률을 바탕으로 올해 성장률 전망치가 조정될 가능성에 대해 이렇게 말했습니다.
성장률 하락의 가장 큰 요인인 수출 감소의 배경에 대해 이 총재는 "금액 기준으로 봐서는 수출이 안 떨어졌는데, 수량을 기준으로 떨어졌다"며 "자동차 파업 등 일시적 요인과 화학제품·반도체의 중국과 경쟁 등으로 수량이 안 늘어나는 것 같은데, 원인을 더 분석해봐야 할 사안"이라고 진단했습니다.
다음 달 28일 열릴 기준금리 결정 방향에 대해서는 "금리 결정할 때 하나의 변수만 보지 않고 종합적으로 보는데, 우선 미국 대선과 연방준비제도 금리 결정으로 경제 상황이 어떻게 변할지 보겠다"고 밝혔습니다.
또 "아울러 이후 달러가 어떻게 될지, 수출 등 내년 경제 전망과 거시안전성 정책이 부동산·가계부채에 미치는 영향 등도 고려해 결정하겠다"고 말했습니다.
'''

system_role = '''당신은 신문기사에서 핵심을 요약하는 어시스턴트입니다.
응답은 다음의 형식을 지켜주세요
{"summary": \"텍스트 요약\"}
'''

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_role
        },
        {
            "role": "user",
            "content": input_text
        }
    ]
)

# 답변
answer = response.choices[0].message.content
print(answer)

{"summary": "한국은행 총재가 올해 성장률 전망치를 2.4%보다 낮게 2.2∼2.3%로 조정할 가능성이 크다고 밝혔다. 수출 감소의 주된 이유는 수량이 줄어든 것으로 분석되며, 자동차 파업과 중국과의 경쟁이 영향을 미쳤다고 설명했다. 기준금리 결정 방향에 대해서는 미국 대선과 연방준비제도의 영향을 고려하며, 달러 가치와 내년 경제 전망, 거시안전성 정책의 영향을 종합적으로 고려할 것이라고 밝혔다."}


* 문서 요약 함수로 생성

In [27]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 시스템 역할과 응답 형식 지정
    system_role = '''당신은 응급상황에 대해서 핵심 키워드를 포함하여 한 줄로 요약 해주는 어시스턴트입니다.'''

    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답형식을 정리하고 return
    return response.choices[0].message.content

* 저장된 text를 하나씩 불러와서 요약하고 다시 저장하기

In [28]:
for text in df['text']:
  summary = text_summary(text)
  df.loc[df['text'] == text, 'summary'] = summary

df

,filename,text,summary
0,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...",머리를 박고 지난 시간 경과 후에도 어지러움과 메스꺼움이 지속된다면 즉시 의료진의 ...
1,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,화장실에서 넘어져 엉덩방아를 찧고 꼬리뼈가 계속 아프다면 정확한 상태를 파악하기 위...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"응급 상황 관련 키워드: 코 막힘, 물질 잠금, 동생, 힘들어 함\n\n한 줄 요약..."
3,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,"응급 상황이니까 혹시나 핵심 키워드를 요약하면 ""넘어져서 머리에 피 나면서 어지러워..."
4,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"응급상황: 고열, 두통, 혼란 - 119에 전화하거나 가까운 응급실로 즉시 이동해야..."
5,5등급(2).m4a,저희 할머니가 삿고랑 부위에 통증과 함께 부종이 있다고 하세요. 상태가 심각해 보이...,"삿고랑 부위에 통증과 부종이 있는 경우, 가능한 빠르게 응급실을 방문해 응급 의료진..."
6,5등급(1).m4a,제 조카가 벌에 쏘였는데요 소인 부위가 조금 붓고 아프다고 합니다. 숨 쉬는데 문제...,벌에 쏘인 부위가 붓고 아프면서 호흡에는 문제가 없다는 것으로 추정됩니다.
7,1등급(1).m4a,안녕하세요. 아버지가 방금 전 약을 잘못 드신 것 같아요. 숨쉬는 게 굉장히 힘들어...,응급실로 즉시 이동해야 합니다. 호흡이 어려워하고 있다면 신속한 치료가 필요합니다.
8,2등급(1).m4a,제 어머니가 넘어지면서 눈을 다치셨어요. 부딪힌 후부터 시야가 흐릿해졌다고 하셔서 ...,"응급: 넘어짐, 눈 다침, 시야 흐릿. 관련 의료진과 상담이 필요합니다."
9,3등급(1).m4a,저희 아버지가 갑자기 한쪽 귀가 안 들린다고 하세요. 어제까지는 괜찮으셨는데 오늘 ...,급성 난청 가능성으로 응급 상황이므로 즉시 전문의 상담이 필요합니다.


## **4. 전국 병원 응급실 정보 수집**



#### 1) 인증키 발급

* 인증키 발급 절차
    * 1) data.go.kr 로그인인
    * 2) 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
https://www.data.go.kr/data/15000563/openapi.do 로 이동
    * 3) 활용신청
        * 활용목적 : 기타(개인 학습 용도)
        * 상세 기능선택
            * 응급의료기관 목록정보 조회
            * 응급의료기관 위치정보 조회
            * 응급의료기관 기본정보 조회
    * 4) 인증키 확인
        * 마이페이지 > Open API > 활용신청현황
        * [승인] 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
        * 일반 인증키(Decoding) 이용

#### 2) 데이터 수집

In [29]:
# path 확인
path

'/content/drive/MyDrive/project6_2/'

In [31]:
# 응급실 데이터 수집하기

url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire'
serviceKey = 'hWmF4Oa3oooGh03of+rgn/B9+LOuDGVESUXnimovPlPGi/MewzgaScWYnMpc01QuesqAJwGdCEuvbFalD8CjTg=='  # 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [41]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
    duty_name = item.findtext('dutyName')
    duty_addr = item.findtext('dutyAddr')
    duty_lat = item.findtext('wgs84Lat')
    duty_long = item.findtext('wgs84Lon')
    duty_tel = item.findtext('dutyTel1')

    # 빈 리스트 data에 딕셔너리 형태({'칼럼이름':값, ...})로 저장(추가)
    data.append({'응급실 이름': duty_name, '응급실 주소': duty_addr, '위도': duty_lat, '경도': duty_long, '전화번호': duty_tel})

# 데이터프레임으로 변환
emergency_room = pd.DataFrame(data)
emergency_room.head()

,응급실 이름,응급실 주소,위도,경도,전화번호
0,경희대학교병원,서울특별시 동대문구 경희대로 23 (회기동),37.5938765502235,127.05183223390303,02-958-8114
1,건국대학교병원,서울특별시 광진구 능동로 120-1 (화양동),37.54084479467721,127.0721229093036,1588-1533
2,중앙대학교병원,서울특별시 동작구 흑석로 102 (흑석동),37.50707428493414,126.96079378447554,1800-1114
3,순천향대학교 부속 서울병원,서울특별시 용산구 대사관로 59 (한남동),37.53384172231443,127.00441798640304,02-709-9114
4,이화여자대학교의과대학부속목동병원,서울특별시 양천구 안양천로 1071 (목동),37.53654282637804,126.8862159683056,02-2650-5114


In [42]:
# csv 파일로 저장(인덱스 제외)
emergency_room.to_csv('emergency_room.csv', index=False)

In [43]:
pd.read_csv('emergency_room.csv')

,응급실 이름,응급실 주소,위도,경도,전화번호
0,경희대학교병원,서울특별시 동대문구 경희대로 23 (회기동),37.593877,127.051832,02-958-8114
1,건국대학교병원,서울특별시 광진구 능동로 120-1 (화양동),37.540845,127.072123,1588-1533
2,중앙대학교병원,서울특별시 동작구 흑석로 102 (흑석동),37.507074,126.960794,1800-1114
3,순천향대학교 부속 서울병원,서울특별시 용산구 대사관로 59 (한남동),37.533842,127.004418,02-709-9114
4,이화여자대학교의과대학부속목동병원,서울특별시 양천구 안양천로 1071 (목동),37.536543,126.886216,02-2650-5114
...,...,...,...,...,...
995,수연세안과의원,"서울특별시 서초구 서초대로77길 54, 서초더블유타워 5, 6층 (서초동)",37.502506,127.024841,02-2258-0077
996,서초좋은의원,"서울특별시 서초구 서초중앙로 238, 306호 (반포동, 삼호가든상가)",37.502962,127.012071,02-591-3600
997,베스탑비뇨기과의원,"서울특별시 송파구 올림픽로 269, 2층 221호 (신천동, 롯데캐슬골드)",37.514427,127.100611,02-416-4747
998,연세이김마취통증의학과의원,서울특별시 송파구 송파대로30길 5 (가락동),37.494481,127.118048,02-400-7240
